build a tree to classify severity of accidents based on conditions

- choose feature to be common conditions people face when driving
- try different trees with specific different features and random forest with random different features
- maybe look at combinations of weather/infrastructure things
- what does the difference tell us?